In [4]:
import sys
sys.path.append('/home/michael/fpga_wsp/litex_test_project/vc707_gt_test/spi')
from ad9174 import Ad9174Settings
from transport import TransportLayer
from numpy import arange
from random import randint

# Print output lane format
for comparison against __Table 17 - Table 21__ in [AD9174 datasheet](https://www.analog.com/media/en/technical-documentation/data-sheets/AD9174.pdf)

```
Upper byte: converter index + 0x0A
Lower byte: sample index

for example 0c 01 = second sample of third converter
```

In [45]:
for m in [*range(13), *range(18, 23)]:
    print('\n----------------')
    print(' mode', m)
    print('----------------')
    settings = Ad9174Settings(m)
    settings.calc_fchk()
    print(settings)
    tl = TransportLayer(settings)

    # test data:
    # upper byte: converter index + 0x0A
    # lower byte: sample index
    samples = []
    for m in range(settings.M):
        samples.append(list((0x0A + m << 8) | arange(16)))  # sample data = index values
#         samples.append([randint(0, 2**settings.N - 1) for _ in range(32)])  # sample data = random values

    # res[lane][frame][octet]    
    res = tl.encode(samples)

    # print transport layer encoded output
    for f in range(2):
        print('\nframe {:d}'.format(f))
        for l in range(settings.L):
            print('l {:d}: '.format(l), end='')
            for o in res[l][f]:
                print('{:02x} '.format(o), end='')
            print()
    
    # decode again to get back (hopefully) the original test data
    print()
    samples_ = tl.decode(res)
    if samples == samples_:
        print('decode equal')
    else:
        print('decode not equal')
        print(samples)
        print(samples_)
        break



----------------
 mode 0
----------------
JESD204BSettings(): 00 00 00 80 03 1f 01 0f 2f 20 80 00 00 81 
         DID:   0        BID:   0     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   1        SCR:   1 
           F:   4          K:  32          M:   2          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   1 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 129 

frame 0
l 0: 0a 00 0b 00 

frame 1
l 0: 0a 01 0b 01 

decode equal

----------------
 mode 1
----------------
JESD204BSettings(): 00 00 00 81 03 1f 03 0f 2f 20 80 00 00 84 
         DID:   0        BID:   0     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   2        SCR:   1 
           F:   4          K:  32          M:   4          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   1 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:  